![MIoT_GDPI](img/MIOT_GDPI_header.png)

# Unidad 05 - Introducción a las Redes Neuronales Recurrentes (RNN)
Típicamente, las plantas industriales cuentan con líneas de producción secuenciales en las que los procesos son altamente no lineales y tienen grandes dependencias temporales. Como consecuencia, los modelos predictivos que no tienen en cuenta el tiempo, suelen ser poco apropiados. En esta práctica introduciremos las Redes Neuronales Recurrentes (RNN) simples como sistemas capaces de modelar procesos en los que se tiene en cuenta el histórico de las observaciones para poder predecir con precisión. 



La mayor parte del contenido de este Notebook se dedica a explicar la adaptación de los datos y el uso del API Keras para generar y utilizar RNNs. Es crucial que dediquéis tiempo a leer y comprender el material, en lugar de simplemente ejecutar el código. Os invitamos a experimentar modificando y variando el código proporcionado para que podáis explorar las distintas opciones y profundizar en cada uno de los conceptos mostrados.



**Importante**: El Notebook contiene varios ejercicios sencillos que deberéis desarrollar durante la clase y enviarlos por el aula virtual del curso a través de la tarea correspondiente.




## Referencias útiles para la práctica

1. [Documentación oficial](https://www.tensorflow.org/learn?hl=es-419) de Tensorflow
2. [Guías](https://keras.io/guides/) de Keras
3. [Vídeo](https://www.youtube.com/watch?v=AsNTP8Kwu80 ) intuitivo y ameno sobre las RNNs
4. A. Bosch Rué, J. Casas-Roma, T. Lozano Bagén (2019): [Deep learning : principios y fundamentos](https://elibro-net.ezbusc.usc.gal/es/ereader/busc/126167/).




## 1. Redes Neuronales Recurrentes
En el ámbito de la optimización y el análisis predictivo dentro de las líneas de producción industrial, nos encontramos a menudo con procesos que no son estáticos, sino que se desarrollan a lo largo del tiempo, donde cada paso o estación influye en los siguientes. La naturaleza secuencial de estas operaciones es fundamental: el estado actual de un producto o del proceso mismo depende intrínsecamente de su historia previa. Si intentamos aplicar modelos de aprendizaje automático que tratan cada punto de datos o cada instante como una observación independiente, sin tener en cuenta el orden o la dependencia temporal (como haríamos con modelos tradicionales que asumen independencia y distribución idéntica), estaremos ignorando información crucial. Estos modelos no pueden "recordar" lo que ocurrió antes, lo que los hace inadecuados para capturar patrones que se despliegan a lo largo de la secuencia de producción. Es aquí donde las Redes Neuronales Recurrentes (RNNs) cobran relevancia.

Su arquitectura inherente, diseñada para procesar secuencias, les permite mantener un estado interno que actúa como una memoria, incorporando información de pasos temporales anteriores para influir en el procesamiento del paso actual. Esta capacidad de aprender y utilizar el contexto histórico las convierte en la herramienta perfecta para modelar las complejas dinámicas temporales de una línea de producción secuencial, permitiéndonos abordar tareas como la detección temprana de fallos que se desarrollan con el tiempo, la predicción de desvíos en variables de calidad teniendo en cuenta las observaciones históricas  o la optimización de parámetros en tiempo real considerando el impacto de ajustes pasados.


### 1.1 Concepto de recurrencia

Hasta ahora, en todas las ANN que habéis visto la información fluye siempre en una única dirección: desde las neuronas de entrada a las neuronas de salida. Son las ANN conocidas como *feedforward*, pero, en realidad, nada impide que haya conexiones desde una capa posterior a una anterior. Empleando esta particularidad podemos hacer que cuando la ANN procese una observación concreta, utilice también información que generó la red en un paso anterior. 


In [ ]:

# Importaciones generales
try:
    import pandas as pd
except ImportError as err:
    !pip install pandas
    import pandas as pd

try:
    import numpy as np
except ImportError as err:
    !pip install numpy
    import numpy as np


try:
    import seaborn as sns
except ImportError as err:
    !pip install seaborn
    import seaborn as sns



try:
    import matplotlib.pyplot as plt
except ImportError as err:
    !pip install matplotlib
    import matplotlib.pyplot as plt

import os

# Asegurarnos de usar Keras 3 con backend TensorFlow
# Es necesario hacerlo antes de cargar Keras
os.environ["KERAS_BACKEND"] = "tensorflow"

# Importaciones de Keras y TensorFlow
try:
    import keras
except ImportError as err:
    !pip install keras
    import keras

try:
    import tensorflow as tf 
except ImportError as err:
    !pip install tensorflow
    import tensorflow as tf 



# Importaciones de Scikit-learn
#Solo para facilitarnos el uso de algunas operaciones típicas
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Configuraciones para visualización. No es necesario
plt.style.use('seaborn-v0_8-whitegrid') # Estilo de gráficos
sns.set_palette('viridis') # Paleta de colores


# Verificación de versiones
#Existen diferentes versiones y compatibilidades. Es importantes saber lo que estamos ejecutando
#Para estos ejemplos queremos ejecutar el API de Keras3 (standalone) y tensorflo3>2.16
print(f"Versión de Keras: {keras.__version__}")
print(f"Backend de Keras: {keras.backend.backend()}")
print(f"Versión de TensorFlow: {tf.__version__}")



# Verificar si TensorFlow puede usar GPU en nuestro sistema (opcional, pero bueno saberlo)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"GPU disponible: {gpu_devices}")
else:
    print("GPU no encontrada, se usará CPU.")


#Establecemos una semilla para todos los procesos en los que lo necesitemos
SEED=1234

print(f"la semilla que emplearemos para todos los procesos pseudoaleatorios es:{SEED}")

# Establece la semilla para los números aleatorios con  keras.utils.set_random_seed. Esto establecerá:
# 1) `numpy` seed
# 2) backend random seed
# 3) `python` random seed
#Permitir la reproducibilidad es clave para poder comparar
keras.utils.set_random_seed(SEED)

## 2. Generación de un *dataset* sintético

Vamos a crear un dataset sintético  que imite una fábrica con una línea de producción  secuencial dividida en **5 secciones**. 

1.  **Sensores en cada sección:**
    * En cada sección, una máquina realiza una operación (ej: apretar un tornillo, calentar una pieza, etc.) y un sensor mide un parámetro clave de esa operación (ej: las revoluciones por minuto de un motor, la temperatura aplicada, etc.). Tendremos 5 series de datos, una por cada sensor de sección.
    * Cada sección puede influir en la siguiente; por ejemplo, si una máquina en la sección 2 trabajó muy rápido, la sección 3 podría tener que ajustarse ligeramente.

2.  **Variable de Calidad Final:**
    * Al final de toda la línea, se inspecciona el producto y se mide una **variable de calidad** (ej: la resistencia de una soldadura, la humedad interna, el perfil de densidad, etc).
    * Esta calidad final **NO depende solo de la última operación**. Más bien, es el resultado de una *combinación* de cómo se aplicaron los procesos previos en las diferentes secciones cuando ese producto pasó por ellas.
        * Por ejemplo, la calidad podría depender mucho de lo que hizo la máquina de la sección 1 cuando el producto estuvo allí (hace 5 "pasos de tiempo"), un poco de la sección 2 (hace 3 "pasos de tiempo"), y así sucesivamente. Algunas secciones podrían no influir nada en esta variable de calidad particular.
    * Esta relación entre las operaciones pasadas y la calidad final no es simple (es no-lineal) y tiene algo de "ruido" o variabilidad, como en la vida real.

3.  **El Reto para la Red Neuronal:**
    * Nuestro DataFrame mostrará, para cada producto, las lecturas actuales de los 5 sensores y su calidad final.
    * La Red Neuronal Recurrente (RNN) recibirá secuencias de las lecturas de los sensores de varios productos consecutivos. Su tarea será **aprender y descubrir por sí misma cómo esas lecturas pasadas (y de qué secciones específicas y con qué "retraso") afectan a la calidad del siguiente producto**, sin que le digamos explícitamente "fíjate en la sección 1 de hace 5 pasos".

Este dataset nos permitirá simular un problema realista donde es necesario analizar secuencias temporales para predecir un resultado futuro que tiene dependencias complejas con el pasado.

In [ ]:
# ## Generación de Datos Sintéticos (Línea de Producción Secuencial)

N_SAMPLES_FINAL = 20000 # Número de muestras finales en nuestro dataset
N_SECTIONS = 5 #secciones de nuestra línea de producción
SEQUENCE_LENGTH = 5 # Usaremos secuencias de 5 timesteps para predecir el siguiente.

# Parámetros para la generación de datos de sensores por sección
# Lags (en pasos de tiempp para la contribución de cada sección a la calidad final.
# L_s = k significa que el valor del sensor de la sección 's' hace k observaciones afectó la calidad del producto actual.
# Ejemplo: L_s = 1 significa que P_{s, t-1} afecta a Q_t.
section_lags = np.array([5, 4, 3, 1, 1]) # Lags para S1, S2, S3, S4, S5 respectivamente

L_max = np.max(section_lags) if len(section_lags) > 0 else 0

# Necesitamos generar más datos al principio para acomodar los retardos máximos.
N_RAW_SAMPLES = N_SAMPLES_FINAL + L_max

# Inicializar arrays para los valores de los sensores
P_raw = np.zeros((N_SECTIONS, N_RAW_SAMPLES))
time_raw = np.arange(N_RAW_SAMPLES)#pasos de tiempo

# Parámetros base para cada sección (amplitud, frecuencia, offset, ruido)
base_params_amplitude = [10, 8, 12, 7, 10]
base_params_period = [50, 180, 10, 70, 45]
base_params_offset = [100, 80, 120, 90, 110]
base_params_noise_std = [2, 1.5, 2, 1, 1.8]



# Sección 1 (S1) - Comportamiento base
P_raw[0, :] = 100 + 10 * np.sin(time_raw / 50) + 5 * np.cos(time_raw / 20) + np.random.randn(N_RAW_SAMPLES) * 2
P_raw[0, :] = (base_params_amplitude[0] * np.sin(time_raw / base_params_period[0])) +  base_params_offset[0] + \
                5 * np.cos(time_raw / 20) + np.random.randn(N_RAW_SAMPLES) * base_params_noise_std[0]

# Secciones 2 a 5 (S2-S5) - Influenciadas por la sección anterior
# Coeficientes de influencia de la sección anterior (pequeños)
alpha_influence = [0.15, -0.2, 0, 0.05] # Para S2 por S1, S3 por S2, etc.




for s in range(1, N_SECTIONS): # Para S2, S3, S4, S5
    influence_term = alpha_influence[s-1] * P_raw[s-1, :]
    base_behavior = (base_params_amplitude[s-1] * np.sin(time_raw / base_params_period[s-1]) +
                     base_params_offset[s-1] +
                     np.random.randn(N_RAW_SAMPLES) * base_params_noise_std[s-1])
    P_raw[s, :] = influence_term + base_behavior






# Generación de la Variable de Calidad Final (Q_t)
# Q_t = f(P_{1,t-L1}, P_{2,t-L2}, P_{3,t-L3}, P_{4,t-L4}, P_{5,t-L5}) + global_noise_t
# Los pesos (w_s) indican la contribución de cada sección. Algunos pueden ser cero.
quality_weights = np.array([0.8, 0.2, -0.5, 0.1, 0]) 



Q_raw = np.zeros(N_RAW_SAMPLES)
base_quality_offset = 60  #valor base de la variable de calidad al que se le añade el efecto de las otras secciones
#quality_noise_std = 1.5



#Generamos una variable de calidad con dependencias temporales
#pero con un comportamiento lineal. Las redes RNN tienen sus limitaciones
#generar un 
for t in range(L_max, N_RAW_SAMPLES):   
    Q_raw[t] = base_quality_offset
    for i in range(N_SECTIONS):
        Q_raw[t] += quality_weights[i] * P_raw[0, t - section_lags[i]] #podríamos crear un comportamiento lineal
    
    #np.random.randn(N_RAW_SAMPLES) + quality_noise_std #Podemos complicarlo añadiendo ruido
    
    

    
    
#Generamos un dataframe por compdidad

df_cols = {}
for s in range(N_SECTIONS):
    df_cols[f'Seccion_{s+1}'] = P_raw[s, L_max:]
df_cols['Calidad'] = Q_raw[L_max:]

data_df = pd.DataFrame(df_cols)

print("Primeras filas del dataset generado:")
print(data_df.head())
print(f"Estructura del dataset: {data_df.shape}")

    

In [ ]:
# ## 3. Exploración y Visualización de Datos
# Visualizar las series temporales generadas para entender sus patrones.

fig, axs = plt.subplots(N_SECTIONS + 1, 1, figsize=(14, 18), sharex=True)
features_to_plot = [f'Seccion_{s+1}' for s in range(N_SECTIONS)] + ['Calidad']

for i, feature_name in enumerate(features_to_plot):
    axs[i].plot(data_df[feature_name], label=feature_name)
    axs[i].set_ylabel(feature_name)
    axs[i].legend(loc="upper right")
axs[-1].set_xlabel("Tiempo")
fig.suptitle("Series Temporales de Parámetros de Sección y Calidad del Producto", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

## 3. Preparación de los datos




In [ ]:
# Características de entrada (X): S1_Param, S2_Param, S3_Param, S4_Param, S5_Param en el instante t.
# Variable objetivo (Y): Calidad en el instante t
def separar_inputs_outputs(dataset):
    """Separa un DataFrame de Pandas en características (inputs) y la columna objetivo 'Calidad'.
    Args:
        dataset (pd.DataFrame): El DataFrame de Pandas que se va a procesar.
            Se espera que este DataFrame contenga las características de entrada
            y, potencialmente, la columna objetivo 'Calidad'.

    Returns:
          Una tupla con dos elementos:
            1. inputs (pd.DataFrame | None): Un DataFrame con las características
               (todas las columnas del dataset original excepto 'Calidad').
               Retorna `None` si la columna 'Calidad' no se encuentra en el
               `dataset`.
            2. outputs (pd.Series | None): Una Serie de Pandas con la columna
               'Calidad'. Retorna `None` si la columna 'Calidad' no se
               encuentra en el `dataset`.
    """
    if 'Calidad' in dataset.columns: 
        return dataset.drop('Calidad', axis=1), dataset['Calidad']
    else: return None, None

def separar_conjuntos(dataset, perc_entrenamiento=0.7):
    """Divide un conjunto de datos secuencial en entrenamiento, validación y prueba.
    - El conjunto de entrenamiento obtiene `perc_entrenamiento` del total de muestras.
    - El porcentaje restante, `(1 - perc_entrenamiento)`, se divide equitativamente
      entre los conjuntos de validación y prueba.
    Args:
        dataset: El conjunto de datos completo a dividir.
        perc_entrenamiento (float, optional): Proporción del dataset a asignar al
            conjunto de entrenamiento. 

    Returns:
        Tuple[Sequence, Sequence, Sequence]: Una tupla con tres subconjuntos de datos:
            1. El subconjunto de entrenamiento.
            2. El subconjunto de validación.
            3. El subconjunto de prueba.
    """
    perc_entrenamiento=perc_entrenamiento if 0<perc_entrenamiento<1 else 0.7
    
    num_train_samples = int(perc_entrenamiento * len(dataset))
    num_val_samples =int((1-perc_entrenamiento)/2 * len(dataset))
    num_test_samples = len(dataset) - num_train_samples - num_val_samples
    return dataset[:num_train_samples], \
            dataset[num_train_samples : num_train_samples + num_val_samples],\
            dataset[num_train_samples + num_val_samples:]
    
    

train_set, val_set, test_set=separar_conjuntos(data_df)


train_features, train_targets=separar_inputs_outputs(train_set)
val_features, val_targets=separar_inputs_outputs(val_set)
test_features, test_targets=separar_inputs_outputs(test_set)

print(f"Estructura de entrenamiento (features): {train_features.shape}")
print(f"Estructura de entrenamiento (outputs): {train_targets.shape}")


#feature_cols = [f'Seccion_{s+1}' for s in range(N_SECTIONS)]
#target_col = 'Calidad'


#rnn_features = data_df[feature_cols].values
#rnn_targets = data_df[target_col].values

#print(f"Forma de rnn_features: {rnn_features.shape}")
#print(f"Forma de rnn_targets: {rnn_targets.shape}")


### 3.1 Estandarización de los datos con Keras


En lugar de estandarizar los datos a través de una operación de `scikit-learn` (sistema que ya conocéis), lo haremos desde el propio Keras generando una [capa de preprocesado](https://keras.io/api/layers/preprocessing_layers/). Esto tiene como ventaja poder incluirlo en el propio modelo y que este sea autosuficiente. Debéis de recordar que el entrenamiento de los modelos es solo la primera fase, luego es necesario desplegarlos en producción y trabajar con ellos. Poder guardar con el modelo toda la información necesaria para poder emplearlo (ej. capa de estandarización), simplifica su empleo.

In [ ]:
# Estandarización de las características de entrada

normalization_layer = keras.layers.Normalization(axis=-1)
normalization_layer.adapt(train_features.values) # ajustar SOLO con datos de entrenamiento (las 5 características)


## 3.2 Preparación de las secuencias de datos
Para poder  entrenar modelos densos recurrentes es necesario preparar las secuencias que vamos a emplear para entrenarlos. Keras espera un tensor de la forma `(batch_size, timesteps, features)`:
* `batch_size` (tamaño del lote): Es el número de secuencias que se procesan simultáneamente en una pasada de entrenamiento.
* `timesteps` (pasos de yiempo o longitud de la secuencia): Es el número de observaciones consecutivas en cada secuencia individual que se van a emplear para entrenar.
* `features` (características): Es el número de variables o características que observas en cada paso de tiempo.

Para preparar las secuencias respecto a nuestros *datasets* emplearemos [keras.utils.timeseries_dataset_from_array](https://keras.io/api/data_loading/timeseries/). Su principal objetivo es facilitar la preparación de datos para modelos que aprenden de secuencias, como las Redes Neuronales Recurrentes (RNNs), LSTMs, GRUs o Transformers, cuando se trabaja con datos de series temporales.




In [ ]:

# **Creación de Secuencias (Ventanas de Tiempo)**
# empleando `tf.keras.utils.timeseries_dataset_from_array`.
# el formato generado es (batch_size, timesteps, features)

def generacion_secuencias_temporales(features, targets, seq_len, solape=False, shuffle=True):
    BATCH_SIZE = 32    
    seq_stride=1 if  solape else seq_len

    return  keras.utils.timeseries_dataset_from_array(
        data=features,
        targets=targets,
        sequence_length=seq_len,
        sequence_stride= seq_stride,#importante decidir si queremos o no secuencias con solape
        batch_size=BATCH_SIZE,
        shuffle=shuffle,
)



train_dataset=generacion_secuencias_temporales(train_features, train_targets, solape=False, seq_len=SEQUENCE_LENGTH)
val_dataset=generacion_secuencias_temporales(val_features, val_targets, solape=False, seq_len=SEQUENCE_LENGTH, shuffle=False)
test_dataset=generacion_secuencias_temporales(test_features, test_targets, solape=False,  seq_len=SEQUENCE_LENGTH, shuffle=False)



print("\nInspección de un batch del dataset de entrenamiento:")
for batch_inputs, batch_targets in train_dataset.take(1):
    print("Forma de las entradas del lote (batch_inputs):", batch_inputs.shape) # (BATCH_SIZE, SEQUENCE_LENGTH, N_SECTIONS)
    print("Forma de las etiquetas del lote (batch_targets):", batch_targets.shape) # (BATCH_SIZE,)

print(f"Batches de entrenamiento: {len(train_dataset)}")
print(f"Batches de validación: {len(val_dataset)}")
print(f"Batches de test: {len(test_dataset)}")




## 4. Definición del modelo 

Desarrollaremos un modelo simple RNN con una capa de entrada, una oculta RNN y una densa de salida. 

In [ ]:
# Definición del Modelo RNN
# Modelo secuencial con: Input, Normalization, SimpleRNN, Dense (salida).

def get_model(seq_len, num_model_features, preprocesing_layers, name_model, type_rnn=True):
    hidden_units=64
    #input layer

    #input layer
    model = keras.Sequential(name=name_model)

    shape=(seq_len, num_model_features) if type_rnn else (num_model_features, )
    model.add( keras.Input(shape=shape, name="input_layer"))

        
    
    
    #preprocesing layers
    for layer in preprocesing_layers:
       model.add(layer)


    #Hidden layer
    hidden_layer= keras.layers.LSTM(units=hidden_units, activation='relu', name="hidden_layer") if type_rnn \
                                    else keras.layers.Dense(hidden_units, activation='relu', name="hidden_layer")

    model.add(hidden_layer)
    model.add( keras.layers.Dense(units=1,  activation='linear', name="output"))

    return model

In [ ]:
rnn_model=get_model(SEQUENCE_LENGTH, N_SECTIONS, [normalization_layer], "rnn_model", type_rnn=True)
rnn_model.summary()

In [ ]:
dense_model=get_model(SEQUENCE_LENGTH, N_SECTIONS, [normalization_layer], "dense_model", type_rnn=False)
dense_model.summary()

### 4. Compilación del modelo


Antes de entrenar, necesitamos "compilar" el modelo para configurar su proceso de entrenamiento. Esto implica definir:

* **Optimizador**: Algoritmo que ajusta los pesos de la red durante el entrenamiento (ej. Adam, SGD, RMSprop). Adam es una opción robusta y popular. Tenéis [aquí](https://keras.io/api/optimizers/) un listado de los disponibles. La configuración de los parámetros del optimizador sería algo para optimizar en la búsqueda hiperparamétrica.

* **Función de Pérdida** (*Loss Function*): Mide qué tan bien se desempeña el modelo durante el entrenamiento. Para regresión, mean_squared_error (MSE) es una opción muy típica. Tenéis [otras](https://keras.io/api/losses/) disponibles.


* **Métricas**: Funciones para evaluar el rendimiento del modelo (ej. mean_absolute_error - MAE). **Las métricas no se usan para optimizar el modelo, solo para monitorizarlo**. Tenéis [aquí](https://keras.io/api/metrics/) un listado de las disponibles

In [ ]:
# Compilación del Modelo
# Optimizador Adam, pérdida MSE, métrica MAE.

rnn_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)



dense_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

### 5 Entrenamiento del modelo

Ya tenemos definido y configurado el modelo para poder entrenarse con nuestro  dataset de entrada. Recordad que el **modelo contiene una capa de normalización**, por lo que **los datos que se le pasen para entrenar NO pueden estar normalizados**(lo hará el propio modelo).


El método `fit` es el empleado para entrenar. Tenéis [aquí](https://keras.io/api/models/model_training_apis/#fit-method) disponible todos los parámetros posibles. En este caso emplearemos:

* **epochs**: Número de veces que el modelo verá el conjunto de datos completo.
* **batch_size**: Número de muestras procesadas antes de actualizar los pesos (entrenamiento con *mini-batches*).
* **validation_data**: conjunto de datos sobre los que validaremos el entrenamiento del modelo. Lo podemos emplear para comparar configuraciones o para parar el entrenamiento de forma prematura (*early_stopping*), lo que puede ayudar a detectar el *overfitting*.

El método `fit` devuelve un objeto *History* que contiene información sobre el proceso de entrenamiento (pérdida y métricas en cada época) que podremos utilizar posteriormente para graficar el proceso de entreno y poder valorar si tenemos *overfitting* o *underfitting*.

En este caso definiremos un *callback* que permita parar el entrenamiento cuando las métricas sobre el conjunto de validación dejen de mejorar y nos quedamos con el mejor modelo visto.

**IMPORTANTE**: cada vez que ejecutéis el entrenamiento, el modelo parte de su último estado. Si queréis volver a entrenarlo desde cero, entonces tenéis que volver a crearlo.

In [ ]:
# Entrenamiento del Modelo
EPOCHS = 1000 # Aumentamos un poco por la posible mayor complejidad
# Considerar EarlyStopping en una implementación real.
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10, # Número de épocas sin mejora antes de detener
    restore_best_weights=True # Restaurar los pesos del modelo de la mejor época
)

rnn_history = rnn_model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    callbacks=[callback_early_stopping] # Añadimos el callback
)




In [ ]:
dense_history = dense_model.fit(
    train_features,
    train_targets,
    epochs=EPOCHS,
    validation_data=(val_features,val_targets),
    callbacks=[callback_early_stopping] # Añadimos el callback
)

### 6.  Evaluación del Modelo

Una vez entrenado el modelo, tenemos que evaluar su rendimento. Lo haremos desde dos puntos de vista:

1. **Visualizando el historial de entrenamiento**: Graficaremos la pérdida y la/s métrica/s disponibles (el MAE en este caso) tanto para el conjunto de entrenamiento como para el de validación a lo largo de las épocas. Esto nos ayuda a diagnosticar problemas como el *overfitting* (cuando la pérdida de validación empieza a aumentar mientras la de entrenamiento sigue bajando) o el *underfitting* (cuando ambas pérdidas son muy altas).
2. **Evaluando en el conjunto de Test**: Usaremos el método `evaluate` con los datos de Test, que el modelo nunca ha visto, para generar la  estimación final e imparcial del rendimiento del modelo.

In [ ]:
def graficas_entrenamiento(hist, model_name=""):
    """Genera y muestra gráficos de pérdida y MAE del entrenamiento de un modelo Keras.

    Esta función toma un objeto `History` de Keras (devuelto por el método `fit()`)
    y grafica dos subplots:
    1.  La pérdida de entrenamiento ('loss') y la pérdida de validación ('val_loss')
        contra el número de épocas.
    2.  El MAE de entrenamiento  y el MAE de validación contra el número de épocas.

    Args:
        history_obj (keras.callbacks.History): El objeto `History` que Keras
            retorna después de entrenar un modelo con `model.fit()`.
        model_name: nombre del modelo que generó los datos históricos de entrenamiento
    """      
    
    hist=pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(hist['epoch'], hist['loss'], label='Pérdida Entrenamiento')
    plt.plot(hist['epoch'], hist['val_loss'], label = 'Pérdida Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida (MSE)')
    plt.title(f'{model_name} - Pérdida (MSE) durante Entrenamiento')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    # Asegúrate de usar el nombre correcto de la métrica (puede variar ligeramente)
    mae_key = 'mae' if 'mae' in hist.columns else list(hist.columns)[1] # Intenta encontrar la clave MAE
    val_mae_key = 'val_' + mae_key
    plt.plot(hist['epoch'], hist[mae_key], label='MAE Entrenamiento')
    plt.plot(hist['epoch'], hist[val_mae_key], label = 'MAE Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Error Absoluto Medio (MAE)')
    plt.title(f'{model_name} - MAE durante Entrenamiento')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

#Graficar el entrenamiento por épocas. Ayuda a valorar el posible (sobre/infra)entrenamiento

graficas_entrenamiento(rnn_history, model_name=rnn_model.name)
graficas_entrenamiento(dense_history,  model_name=dense_model.name)

In [ ]:
def print_test_metrics(model, test_dataset, targets=None):
    
    print(f"Metricas del modelo: {model.name}")
    if targets is None: #se pasan los datos como un dataset de tensorflow con inputs y ouputs juntos (secuencias)
        test_loss, test_mae = model.evaluate(test_dataset, verbose=0)#para las redes recurrentes
    else:
        test_loss, test_mae = model.evaluate(test_dataset, targets,verbose=0)#para las redes densas
        
    print(f"{model.name} - Pérdida (MSE) en Test: {test_loss:.4f}")
    print(f"{model.name} - Error Absoluto Medio (MAE) en Test: {test_mae:.4f}")
    
def print_seq__pred_vs_real(model, test_dataset, targets=None):
    print(f"Comparativa del modelo: {model.name}")
    predictions = model.predict(test_dataset, verbose=0)

    if targets is None: 
        real_test=[batch_targets_test.numpy() for _, batch_targets_test in test_dataset]#devuelve una lista de lista. Los outputs están agrupados en batches de 32
        #print(f"numero de batches: {len(real_test)} - Elementos en el batch: {len(real_test[0])}")
        actual_real_test = np.concatenate(real_test)#contactenamos todas las listas dentro de la lista (numero de batches) en una sola lista
    else:
        actual_real_test=targets


    print(predictions[:10])    
    print(actual_real_test[:10])    
    r2 = r2_score(actual_real_test, predictions)
    #print(f"{model.name} - Coeficiente de Determinación (R²) en el conjunto de Test: {r2:.4f}")
    plot_range = min(300, len(actual_targets_test))

    plt.figure(figsize=(14, 6))
    plt.plot(actual_targets_test[:plot_range], label='Valores Reales ', color='blue', marker='.', linestyle='-')
    plt.plot(predictions[:plot_range, 0], label='Predicciones del Modelo', color='red', marker='x', linestyle='--')
    plt.title(f'{model.name} - R² en el conjunto de Test: {r2:.4f} - Comparación de Calidad Real vs. Predicha (Primeros {plot_range} productos del Test Set)')
    plt.xlabel('Observaciones ordenadas en el tiempo')
    plt.ylabel('Variable de Calidad')
    plt.legend()
    plt.grid(True)
    plt.show()

    

print_test_metrics(rnn_model, test_dataset)
print_seq__pred_vs_real(rnn_model, test_dataset)



print_test_metrics(dense_model, test_features[::], test_targets[::])
print_seq__pred_vs_real(dense_model, test_features[::], test_targets[::])





